# Решающие деревья и решающие леса.

# 1. Решающие деревья

## Задание 1. Решающие деревья на искусственно сгенерированных данных.

Рассмотрим модельную задачу регрессии. Объектами будут являться точки на плоскости (т.е. каждый объект описывается 2 признаками), целевая переменная — расстояние от объекта до точки (0, 0).

In [ ]:
%pylab inline
import numpy as np
import pandas as pd
import pylab as plt

from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier

Напишем вспомогательную функцию, которая будет возвращать решетку для дальнейшей красивой визуализации

In [ ]:
def get_grid(data):
    x_min, x_max = data[:, 0].min() - 1, data[:, 0].max() + 1
    y_min, y_max = data[:, 1].min() - 1, data[:, 1].max() + 1
    return np.meshgrid(np.arange(x_min, x_max, 0.01),
                         np.arange(y_min, y_max, 0.01))

Сгенерируем выборку

In [ ]:
data_x = np.random.normal(size=(100, 2))
data_y = (data_x[:, 0] ** 2 + data_x[:, 1] ** 2) ** 0.5
plt.figure(figsize=(8, 8))
plt.scatter(data_x[:, 0], data_x[:, 1], c=data_y, s=100, cmap='spring')

In [ ]:
from sklearn.tree import DecisionTreeRegressor

Обучим дерево на сгенерированных данных и предскажем ответы для каждой точки решетки

In [ ]:
clf = DecisionTreeRegressor()
clf.fit(data_x, data_y)

xx, yy = get_grid(data_x)
print(np.c_[xx.ravel(), yy.ravel()])

predicted = clf.predict(np.c_[xx.ravel(), yy.ravel()]).reshape(xx.shape)

plt.figure(figsize=(8, 8))
plt.pcolormesh(xx, yy, predicted, cmap='spring')
plt.scatter(data_x[:, 0], data_x[:, 1], c=data_y, s=100, cmap='spring')

Посмотрим как будет выглядеть разделяющая поверхность в зависимости от 
- минимального количества объектов в листе
- максимальной глубины дерева

In [ ]:
plt.figure(figsize=(18, 18))
for i, max_depth in enumerate([1, 2, 4, 6]):
    for j, min_samples_leaf in enumerate([1, 5, 10, 15]):
        clf = DecisionTreeRegressor(max_depth=max_depth, min_samples_leaf=min_samples_leaf)
        clf.fit(data_x, data_y)
        xx, yy = get_grid(data_x)
        predicted = clf.predict(np.c_[xx.ravel(), yy.ravel()]).reshape(xx.shape)
        
        plt.subplot2grid((4, 4), (i, j))
        plt.pcolormesh(xx, yy, predicted, cmap='spring')
        plt.scatter(data_x[:, 0], data_x[:, 1], c=data_y, s=30, cmap='spring')
        plt.title('max_depth=' + str(max_depth) + ', min_samples_leaf: ' + str(min_samples_leaf))

- Как влияет увеличение максимальной глубины и/или уменьшение минимального количества объектов выборки в листе на качество на обучающей выборке? на переобучение?

### Неустойчивость решающих деревьев

Решающие деревья — это алгоритмы, неустойчивые к изменениям обучающей выборки, т.е. при малейших её изменениях итоговый классификатор может радикально измениться.
Посмотрим, как будет меняться структура дерева при обучении на разных 90%-х подвыборках.


In [ ]:
plt.figure(figsize=(20, 6))
for i in range(3):
    clf = DecisionTreeRegressor(random_state=42)

    indices = np.random.randint(data_x.shape[0], size=int(data_x.shape[0] * 0.9))
    clf.fit(data_x[indices], data_y[indices])
    xx, yy = get_grid(data_x)
    predicted = clf.predict(np.c_[xx.ravel(), yy.ravel()]).reshape(xx.shape)

    plt.subplot2grid((1, 3), (0, i))
    plt.pcolormesh(xx, yy, predicted, cmap='winter')
    plt.scatter(data_x[:, 0], data_x[:, 1], c=data_y, s=30, cmap='winter')

## Задание 2. Подбор параметров для решающего дерева в задаче Boston Houses.

Посмотрим на качество дерева в зависимости от параметров на одном из стандартных наборов данных - Бостонском датасете.

In [ ]:
from sklearn.datasets import load_boston

In [ ]:
data = load_boston()
print(data.DESCR)

In [ ]:
X_full = data.data
y_full = data.target

print(X_full[:3])

In [ ]:
X_full.shape

In [ ]:
y_full[:10]

- разобъём все данные на train и test
- будем оценивать качество алгоритма по кросс-валидации

In [ ]:
from sklearn.model_selection import train_test_split
X, X_test, y, y_test = train_test_split(X_full, y_full, test_size=100, 
                                        random_state=241)

In [ ]:
from sklearn.model_selection import KFold, cross_val_score
cv = KFold(X.shape[0], shuffle=True, random_state=241)

In [ ]:
from sklearn.tree import DecisionTreeRegressor
regr = DecisionTreeRegressor(random_state=241)
print(-cross_val_score(regr, X, y, cv=cv, 
                       scoring='neg_mean_squared_error').mean())

In [ ]:
regr.fit(X, y)

Метрика MSЕ имеет не ограничена сверху. Поэтому для оценки качества алгоритма можно также пользоваться метрикой R2 (коэффициент детерминации), так как он не превышает 1 (и чем ближе к 1, тем лучше).

Выведем на экран значение R2 алгоритма ('r2').

In [ ]:
print(cross_val_score(regr, X, y, cv=cv, 
                       scoring='r2').mean())

Для сравнения качества модели при различных наборах параметров или для сравнения моделей на одном датасете можно использовать, как и раньше, MSE.

Будем подбирать параметры решающего дерева по сетке с целью увеличить качество алгоритма. Будем подбирать значения max_features и max_depth.

In [ ]:
from sklearn.metrics import SCORERS
SCORERS.keys()

## Задание. 

С помощью GridSearchCV подберите оптимальную глубину решающего дерева (max_depth) и количество признаков (max_features).

In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid={'max_features': [None, 'log2', 'sqrt'], 
            'max_depth': #your code here}
            
gs = #your code here
gs.fit(X, y)

In [ ]:
means = gs.cv_results_['mean_test_score']
stds = gs.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, gs.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
            % (mean, std * 2, params))

# Задание

Теперь попробуем одновременно подбирать значения max_features, max_depth и min_samples_leaf. Ищите min_samples_leaf в диапазоне range(1,20).

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid=#your code here
gs = #your code here

gs.fit(X, y)

In [ ]:
gs.best_score_, gs.best_params_

Как в данной задаче зависит качество алгоритма от количества параметров, которые мы оптимизируем?

# 2. Решающий лес

Посмотрим, какое качество в задаче Boston Houses можно получить при использовании решающего леса.

In [ ]:
regr = DecisionTreeRegressor(max_depth = 8, min_samples_leaf = 9)
t1 = time.time()
print(-cross_val_score(regr, X, y, cv=cv,
                       scoring='neg_mean_squared_error').mean())
t2 = time.time()
print('time:', t2-t1)

Выведите среднюю квадратичную ошибку на кросс-валидации для решающего леса с количеством деревьев n_estimators: 10, 100, 1000.

*измерьте время обучения алгоритма

In [ ]:
from sklearn.ensemble import RandomForestRegressor
import time

regr = RandomForestRegressor(n_estimators=#your code here)

#your code here

Построим график качества классификации на кросс-валидации в зависимости от числа деревьев.

На каждой итерации цикла обучаем регрессор командой regr = ... и добавляем в список Scores число cross_val_score:
Scores.append(cross_val_score(...))

In [ ]:
from tqdm import tqdm

Ntrees = np.arange(5,120,20)
Scores = []

for elem in tqdm(Ntrees):
    regr = #your code here
    Scores.append(#your code here)
    
plot(Scores)
xlabel('number of trees', fontsize=18)
ylabel('MSE', fontsize=16)
xticks(arange(len(Ntrees)), Ntrees)

Построим график количества времени, потраченного на обучение в зависимости от числа деревьев.

In [ ]:
from tqdm import tqdm

Ntrees = np.arange(5,120,20)
Times = []

for elem in tqdm(Ntrees):
    regr = RandomForestRegressor(n_estimators=elem)
    t1 = time.time()
    s = -cross_val_score(regr, X, y, cv=cv,
                       scoring='neg_mean_squared_error').mean()
    t2 = time.time()
    Times.append(t2-t1)
    
plot(Times)
xlabel('number of trees', fontsize=18)
ylabel('time', fontsize=16)
xticks(arange(len(Ntrees)), Ntrees)